<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


# DAY1

### Learning Objectives
- Describe the foundational components of a neural network
- Implement a Perceptron from scratch in Python

#### Input Layer:

The input Layer is where the feature data from the dataframe are input

#### Hidden Layer:

These are the layer that exist between the input layer and output layer. You cna have one hidden layer or many hidden layers

#### Output Layer:

This is the answer/result of our neurons in our neural netoworks. These ouputs can then be used as inputs for the next layer of neurons or be the final output(s) of the neural network.

#### Neuron:

The neuron recieves inputs, multiplies the inputs by their weights, sums everyhting up, and then applies the activation function to the sum. Usually involves a continuous activation function

#### Weight:

This is the amount or positive or negative effect an input will be associated with the ending output.

#### Activation Function:

The activation function is how the neural network normalizes the results after inputs, weights, and biases have been applied within the neuron.

#### Node Map:

The node maps show how the features of the dataframe or the outputs of upper level neurons are further processed throughout the neural netowork. It shows inputs, outputs, and hidden layers visualized at a high level.

#### Perceptron:

Simply, a perceptron consists of four distinct parts. Uses a binary activation function that is either activate or not, different from a neuron

    Inputs
    Weights
    Weighted Sum
    Activation Function (Output)

Perceptrons classify data into two parts (0,1) most of the time. Perceptrons are also known as Linear Binary Classifiers


#### Inputs -> Outputs
Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?
Your Answer Here

Depending on your network, Inputs and Outputs can range arbitraily. Each input can come from an upper level neuron or the intial inputted values from a dataframe. Each input can be weighted negatively or positvely depending on whether your desired answer needs the neuron to activate negatively or positively depending how your inputted bias has shifted the activation curve up or down.


### Imports

In [112]:
!pip install category-encoders

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import category_encoders as ce

In [114]:
#Load Data
df = sns.load_dataset('tips')
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [115]:
print(df.shape)
df.head()

(244, 7)


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [116]:
def prep(df, target):
    
    """
    This function will:
    1. Change "size" into a catagorical to be one hotted
    2. Add Total and Tip and put into 3 bins
    3. Split data
    4. Create X and y train/test
    5. process X train/test data by one hotting categoricals
    6. Make 'sex' a binary column
    7. return 4 df's
    """
    df['size'] = df['size'].astype(str)
    df['bill_tip_sum'] = pd.qcut(df['total_bill']+df['tip'], 3, labels=['low', 'medium', 'high'])
    df['tip_pct'] = df['tip']/df['total_bill']
    
    training, testing = train_test_split(df, test_size=.2)
    
    X_train = training.drop(columns=target)
    y_train = training[target]
    X_test = testing.drop(columns=target)
    y_test = testing[target]
    
    processor = make_pipeline(
        ce.OneHotEncoder(use_cat_names=True),  
#         SimpleImputer(strategy='median'), # Use when a normalized dataframe is needed
#         StandardScaler() # Use when a normalized dataframe is needed
    )
    
    gender = {'Female': 0, 'Male': 1}
    y_train = y_train.map(gender)
    y_test = y_test.map(gender)
    
    X_process_train = processor.fit_transform(X_train)
    X_process_test = processor.transform(X_test)
    
    return X_process_train,y_train, X_process_test, y_test

In [117]:
X_train, y_train, X_test, y_test = prep(df, 'sex')
print(X_train.shape) 
print(X_test.shape) 
print(y_train.shape) 
print(y_test.shape)
X_train.head()

(195, 20)
(49, 20)
(195,)
(49,)


,total_bill,tip,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner,size_2,size_4,size_3,size_5,size_1,size_6,bill_tip_sum_low,bill_tip_sum_medium,bill_tip_sum_high,tip_pct
199,13.51,2.0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0.148038
47,32.40,6.0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0.185185
35,24.06,3.6,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0.149626
145,8.35,1.5,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0.179641
11,35.26,5.0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0.141804


In [118]:
class NNet:
    def __init__(self):
        
        # Inputs must be == to number of features
        self.inputs = 20
        # Only one output node b/c only trying to predict one thing
        self.outputNodes = 1
        
        self.weights = np.random.rand(self.inputs, self.outputNodes)
     
    # Squishify
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    # Create 0 or 1 from prediced activated output
    def binary(self, X):
        binary = self.feed_forward(X)
        binary = [1 if x > .9999 else 0 for x in binary]
        return binary
    
     
    def feed_forward(self, X):
        """Calculate the NNet inference using the feed forward, aka predict """
        
        # Combining  inputs and weights in a weighted sum
        self.input_sum = np.dot(X, self.weights)
        
        # Apply activation function to the weighted sum
        self.output_activated = self.sigmoid(self.input_sum)
        
        return self.output_activated

In [119]:
nn = NNet()

In [120]:
y_pred1 = nn.binary(X_train)
score = accuracy_score(y_train, y_pred1)

y_pred2 = nn.binary(X_test)
score2 = accuracy_score(y_test, y_pred2)

print(f"Mean baseline for our target(Males) is {round(df['sex'].value_counts(normalize=True)[0]*100, 2)}%")
print(f"The accuracy of the train is {round(score*100, 2)}%")
print(f"The accuracy of the test is {round(score2*100, 2)}%")

Mean baseline for our target(Males) is 64.34%
The accuracy of the train is 65.13%
The accuracy of the test is 65.31%


# Day 2

### Learning Objectives
- Explain the intuition behind backproprogation
- Implement gradient descent + backproprogation on a feedforward neural network

In [121]:
# I want activations that correspond to negative weights to be lower
# and activations that correspond to positive weights to be higher

class NNetbackprop:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.inputs = 20
        self.hiddenNodes = 3
        self.outputNodes = 1

        # Initial Weights
        # 2x3 Matrix Array for the First Layer
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
       
        # 3x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka "predict"
        """
        
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
        
    def backward(self, X,y,o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        # Calculate the error, the diffrence between true y value and the predicted
        self.o_error = y - o
        
        # Apply Derivative of Sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        # ^- aka hidden => output
        # Which direction do we want to go 
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        # z2 error
        # Applying the  o-delta/correction to weights2 transformed
        self.z2_error = self.o_delta.dot(self.weights2.T)
        
        # How much of that "far off" can explained by the input => hidden
        # Apply sigmoid derivative to the error
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Adjustment to first set of weights (input => hidden)
        # Applying adjustments to the weights
        self.weights1 += X.T.dot(self.z2_delta)
        
        # Adjustment to second set of weights (hidden => output)
        # Applying adjustments to the weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X,y,o)

### Preprocessing

In [122]:
def y_input(x):
    "Changes our y into a list of individual arrays"
    y_list = []
    for x in y_train:
        new = np.array([x])
        y_list.append(new)
    return y_list

In [123]:
nnbp = NNetbackprop()

In [124]:
ytrain = y_input(y_train)
nnbp.train(X_train, ytrain)

### Backproprogation

In [125]:
# ---1st ERROR---
# Apply sigmoid derivative to the error
# Which direction do we want to go
# self.o_delta = self.o_error * sigmoidprime(o)
#How much more sigmoid activation would have pushed us towards the right answer

nnbp.o_error

array([[ 0.20900242],
       [ 0.20900236],
       [ 0.20900236],
       [-0.79099322],
       [-0.79099764],
       [ 0.2090052 ],
       [ 0.20900237],
       [ 0.20900236],
       [ 0.20900238],
       [-0.79099763],
       [ 0.20900236],
       [-0.79099758],
       [ 0.20900236],
       [ 0.20900236],
       [ 0.20900236],
       [-0.79089189],
       [ 0.20900238],
       [-0.79099764],
       [ 0.20900236],
       [-0.79099764],
       [ 0.20900236],
       [-0.79099763],
       [ 0.20900236],
       [ 0.20900236],
       [-0.79099761],
       [-0.79099761],
       [-0.79099762],
       [-0.79099764],
       [ 0.20900236],
       [-0.79099764],
       [ 0.20900236],
       [ 0.20900627],
       [-0.79031213],
       [ 0.20900237],
       [-0.79099764],
       [-0.79099763],
       [ 0.20900245],
       [ 0.20900236],
       [-0.79099764],
       [-0.79099764],
       [ 0.20900241],
       [-0.79099685],
       [ 0.20900236],
       [ 0.20900236],
       [ 0.20900236],
       [-0

In [126]:
# Apply sigmoid derivative to the error
# Which direction do we want to go
# self.o_delta = self.o_error * sigmoidprime(o)

nnbp.o_delta

array([[ 0.03455236],
       [ 0.03455235],
       [ 0.03455235],
       [-0.13076933],
       [-0.13076803],
       [ 0.03455316],
       [ 0.03455235],
       [ 0.03455235],
       [ 0.03455235],
       [-0.13076803],
       [ 0.03455235],
       [-0.13076804],
       [ 0.03455235],
       [ 0.03455235],
       [ 0.03455235],
       [-0.13079921],
       [ 0.03455235],
       [-0.13076803],
       [ 0.03455235],
       [-0.13076803],
       [ 0.03455235],
       [-0.13076803],
       [ 0.03455235],
       [ 0.03455235],
       [-0.13076803],
       [-0.13076803],
       [-0.13076803],
       [-0.13076803],
       [ 0.03455235],
       [-0.13076803],
       [ 0.03455235],
       [ 0.03455347],
       [-0.13096963],
       [ 0.03455235],
       [-0.13076803],
       [-0.13076803],
       [ 0.03455237],
       [ 0.03455235],
       [-0.13076803],
       [-0.13076803],
       [ 0.03455236],
       [-0.13076826],
       [ 0.03455235],
       [ 0.03455235],
       [ 0.03455235],
       [-0

In [127]:
# z2 error
# Applying the  o-delta/correction to weights2 transformed
# These are the errors from the output to the hidden layer

nnbp.z2_error

array([[ 0.01712281,  0.01829274,  0.0105719 ],
       [ 0.0171228 ,  0.01829273,  0.0105719 ],
       [ 0.0171228 ,  0.01829273,  0.0105719 ],
       [-0.06480418, -0.06923201, -0.04001117],
       [-0.06480354, -0.06923132, -0.04001078],
       [ 0.0171232 ,  0.01829316,  0.01057215],
       [ 0.0171228 ,  0.01829274,  0.0105719 ],
       [ 0.0171228 ,  0.01829273,  0.0105719 ],
       [ 0.0171228 ,  0.01829274,  0.0105719 ],
       [-0.06480354, -0.06923132, -0.04001078],
       [ 0.0171228 ,  0.01829273,  0.0105719 ],
       [-0.06480355, -0.06923133, -0.04001078],
       [ 0.0171228 ,  0.01829273,  0.0105719 ],
       [ 0.0171228 ,  0.01829273,  0.0105719 ],
       [ 0.0171228 ,  0.01829273,  0.0105719 ],
       [-0.06481899, -0.06924783, -0.04002032],
       [ 0.0171228 ,  0.01829274,  0.0105719 ],
       [-0.06480354, -0.06923132, -0.04001078],
       [ 0.0171228 ,  0.01829273,  0.0105719 ],
       [-0.06480354, -0.06923132, -0.04001078],
       [ 0.0171228 ,  0.01829273,  0.010

In [128]:
# How much on the sigmoid curve we want to move
# Being the delta this is the direction we will be traveling
# For each observation, how much more sigmoid activation from this layer would have 
# pushed us towards the right answer?

nnbp.z2_delta

array([[ 2.82213443e-09,  1.97829206e-09,  7.11793068e-09],
       [ 0.00000000e+00,  4.06180273e-18,  1.29108812e-16],
       [ 1.34629696e-14,  3.77300856e-14,  1.33527620e-12],
       [-1.47303457e-06, -8.00522201e-07, -1.21848441e-06],
       [-0.00000000e+00, -0.00000000e+00, -1.42146829e-16],
       [ 2.30953635e-07,  4.99057111e-08,  3.11955492e-07],
       [ 3.77780788e-10,  5.16885955e-10,  1.74176250e-09],
       [ 2.12810731e-13,  2.25876850e-13,  3.62757734e-12],
       [ 7.27654390e-10,  2.89470382e-10,  3.50955817e-09],
       [-4.88791121e-10, -6.49987141e-10, -6.99543130e-09],
       [ 3.61874436e-12,  5.25601743e-12,  8.08990076e-11],
       [-1.19433227e-08, -9.95106993e-09, -2.65282434e-08],
       [ 3.84840945e-14,  7.90670520e-14,  1.98340948e-12],
       [ 1.44805018e-11,  9.21216865e-12,  1.37223394e-10],
       [ 1.59685040e-16,  1.14136657e-15,  2.51175326e-14],
       [-2.89156588e-05, -5.92485404e-06, -4.87401276e-05],
       [ 8.08513224e-10,  5.63926552e-10

In [129]:
#Calculation to update the weights
X_train.T.dot(nnbp.z2_delta)

,0,1,2
total_bill,-8.669517e-04,-3.819881e-04,-1.028224e-03
tip,-2.491814e-04,-1.128030e-04,-2.799060e-04
smoker_Yes,-2.436398e-04,-1.079564e-04,-2.696525e-04
smoker_No,-5.476197e-06,-4.438130e-06,-9.721217e-06
day_Thur,-1.959782e-06,-1.498097e-06,-2.356769e-06
day_Fri,-2.900559e-05,-5.800273e-06,-4.910028e-05
day_Sat,-2.183930e-04,-1.051281e-04,-2.279937e-04
day_Sun,2.423570e-07,3.190668e-08,7.696697e-08
time_Lunch,-2.009212e-06,-1.366094e-06,-2.575745e-06
time_Dinner,-2.471068e-04,-1.110284e-04,-2.767980e-04


In [130]:
# Update hidden layer weights

nnbp.activated_hidden.T.dot(nnbp.o_delta)

array([[-4.83444101],
       [-4.8347326 ],
       [-4.83402782]])

In [131]:
# Train my 'net
nnbp = NNetbackprop()

# Number of Epochs / Iterations
for i in range(10000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X_train)
        print('Actual Output: \n', ytrain)
        print('Predicted Output: \n', str(nn.feed_forward(X_train)))
        print("Loss: \n", str(np.mean(np.square(ytrain - nnbp.feed_forward(X_train)))))
    nnbp.train(X_train,ytrain)

+---------EPOCH 1---------+
Input: 
      total_bill   tip  smoker_Yes  smoker_No  day_Thur  day_Fri  day_Sat  \
199       13.51  2.00           1          0         1        0        0   
47        32.40  6.00           0          1         0        0        0   
35        24.06  3.60           0          1         0        0        1   
145        8.35  1.50           0          1         1        0        0   
11        35.26  5.00           0          1         0        0        0   
6          8.77  2.00           0          1         0        0        0   
193       15.48  2.02           1          0         1        0        0   
15        21.58  3.92           0          1         0        0        0   
8         15.04  1.96           0          1         0        0        0   
162       16.21  2.00           0          1         0        0        0   
63        18.29  3.76           1          0         0        0        1   
121       13.42  1.68           0          1       

# Day 3

### Learning Objectives
- Introduce the Keras Sequential Model API
- Learn How to Select Model Architecture
- Discuss the trade-off between various activation functions

In [132]:
#Model
model = Sequential()

#Input
model.add(Dense(16, input_dim=20, activation='relu'))

#Hidden
model.add(Dense(32, kernel_initializer='normal', activation='relu'))
model.add(Dense(32, kernel_initializer='normal', activation='relu'))

#Output
model.add(Dense(1, activation='linear'))

#Compile
model.compile(loss='mean_squared_error',
              metrics=['mean_squared_error'],
              optimizer='adam')

#Fit & Evaluate
history = model.fit(X_train, y_train, epochs=100, verbose=False, validation_split=.1)
scores = model.evaluate(X_test, y_test, verbose=0)

In [133]:
print(f"The MSE of our neural net is ${scores[1]}")
print(f"The RMSE of our neural net is ${round(np.sqrt(scores[1]), 2)}")

The MSE of our neural net is $0.2639111578464508
The RMSE of our neural net is $0.5099999904632568


### Activation Functions

#### Step Function

- Binary activation function. 
- POS activation function

#### Linear Function

- Passes the signal onto the enxt layer by a constant factor
- Derivative is of a linear activation is horizontal which would mean we should update all weights by a constant
amount everytime
- Only used for very simple tasks where intepretability is important

#### Sigmoid Function

- Sigmoid and its derivatives are usually better at classification problems
- Great activation function since its continuously differentiable
- Slope reaches 0 quickly when moving away from zero
- Higher slope around 0 pushes our y more quickly to one of the extremes
- Useful for binary classification

#### Tanh Function

- Doesnt get so flat when moving away from zero
- A little steeper in the middle than sigmoid
- Created by scaling y by 2 in the y dimmension and subtracting 1 from all values
- Same drawbacks as sigmoid like diminishing flat gradients when moving from zero
- Derivative higher at 0 causing weights to move to extremes faster

#### ReLu Function (Rectified Linear Units)

- Typicaly see ReLu for initial layers
- Generally better at obtaining optimal model fit
- Commonly used as activatoin functions in neural networks
- Doesnt activate when neuron outputs a negative signal but passes on positive signal
- Derivative looks like a step function
- Turns off a portion of our less important neurons which decreases computational load
- Can lead to dead neurons since negatively weighted neurons wont activate
- Might want to to upadte negatively initialized weights

#### Leaky ReLu

- Avoids having a gradient of 0 on the left side of the derivative funcion
- Even 'dead' neurons have a chance of being revived w/ enough iterations
- Leaky side slop can even be tuned as a hyper parameter

#### Softmax Funtion

- Good for multiclassification problems
- Takes any set of inputs and translates them into probabilities that sum to 1
- Can ouput a list of outputs and translate them into probabilities that sum to 1

# DAY 4

### Learning Objectives
- Describe the major hyperparameters to tune
- Implement an exeriment tracking framework
- Search the hyperameter space using RandomSearch (Optional)

In [100]:
# Important Hyperparameters
inputs = X_train.shape[1]
epochs = 75
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Fit Model
model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Train on 195 samples, validate on 49 samples
Epoch 1/75
195/195 [==============================] - 0s 1ms/sample - loss: 0.4007 - mean_squared_error: 0.4007 - mean_absolute_error: 0.5426 - val_loss: 0.2367 - val_mean_squared_error: 0.2367 - val_mean_absolute_error: 0.4506
Epoch 2/75
195/195 [==============================] - 0s 152us/sample - loss: 0.2355 - mean_squared_error: 0.2355 - mean_absolute_error: 0.4447 - val_loss: 0.2104 - val_mean_squared_error: 0.2104 - val_mean_absolute_error: 0.4191
Epoch 3/75
195/195 [==============================] - 0s 146us/sample - loss: 0.2012 - mean_squared_error: 0.2012 - mean_absolute_error: 0.3993 - val_loss: 0.2309 - val_mean_squared_error: 0.2309 - val_mean_absolute_error: 0.4488
Epoch 4/75
195/195 [==============================] - 0s 143us/sample - loss: 0.1844 - mean_squared_error: 0.1844 - mean_absolute_error: 0.3824 - val_loss: 0.2304 - val_mean_squared_error: 0.2304 - val_mean_absolute_error: 0.4388
Epoch 5/75
195/195 [=================

In [106]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Replaced with tips dataset
# # load dataset
# url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# dataset = pd.read_csv(url, header=None).values

# # split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.5948718190193176 using {'batch_size': 10, 'epochs': 20}
Means: 0.5948718190193176, Stdev: 0.0691832654313919 with: {'batch_size': 10, 'epochs': 20}
Means: 0.5794872045516968, Stdev: 0.05936326262982861 with: {'batch_size': 20, 'epochs': 20}
Means: 0.5589743852615356, Stdev: 0.014504753621327159 with: {'batch_size': 40, 'epochs': 20}
Means: 0.5794872045516968, Stdev: 0.026148816459833527 with: {'batch_size': 60, 'epochs': 20}
Means: 0.5589743852615356, Stdev: 0.0725237681066358 with: {'batch_size': 80, 'epochs': 20}
Means: 0.5589743753274282, Stdev: 0.09428091042619949 with: {'batch_size': 100, 'epochs': 20}


In [108]:
# define the grid search parameters
param_grid = {'batch_size': [20],
              'epochs': [20, 40, 60,200]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.6410256624221802 using {'batch_size': 20, 'epochs': 20}
Means: 0.6410256624221802, Stdev: 0.07977614491729938 with: {'batch_size': 20, 'epochs': 20}
Means: 0.5948718190193176, Stdev: 0.03161237761816035 with: {'batch_size': 20, 'epochs': 40}
Means: 0.5743589997291565, Stdev: 0.047557015090999834 with: {'batch_size': 20, 'epochs': 60}
Means: 0.6153846283753713, Stdev: 0.0905821708302442 with: {'batch_size': 20, 'epochs': 200}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?